In [ ]:
'''
!pip install requests
!pip install bs4
!pip install selenium
'''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 라이브러리

In [ ]:
import pandas as pd
import time
import json
import os
import requests
import tempfile
import uuid
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time
import json
import os
from datetime import datetime

### 크롤러

In [ ]:
# 네이버 마이비즈 정책지원금 크롤링
# 필요한 라이브러리 설치: pip install selenium beautifulsoup4 pandas requests



class NaverMybizCrawler:
    def __init__(self):
        self.base_url = "https://mybiz.pay.naver.com"
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.session = requests.Session()
        self.session.headers.update(self.headers)

    def setup_selenium(self, headless=True):
        """Selenium 브라우저 설정 - 코랩 환경 최적화 (수정)"""
        from selenium.webdriver.chrome.options import Options
        from selenium.webdriver import Chrome
        from selenium.webdriver.common.by import By
        from selenium.webdriver.support.ui import WebDriverWait
        from selenium.webdriver.support import expected_conditions as EC
        import tempfile
        import uuid

        chrome_options = Options()

        # 코랩 환경 필수 설정
        chrome_options.add_argument("--headless")  # 코랩에서는 항상 headless
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-plugins")
        chrome_options.add_argument("--disable-images")  # 이미지 로딩 비활성화로 속도 향상

        # 사용자 데이터 디렉토리 충돌 해결
        temp_dir = tempfile.mkdtemp()
        unique_user_data_dir = f"{temp_dir}/chrome_user_data_{uuid.uuid4().hex[:8]}"
        chrome_options.add_argument(f"--user-data-dir={unique_user_data_dir}")

        # 자동화 감지 방지
        chrome_options.add_argument("--disable-blink-features=AutomationControlled")
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)

        # 메모리 사용량 최적화
        chrome_options.add_argument("--memory-pressure-off")
        chrome_options.add_argument("--max_old_space_size=4096")

        try:
            self.driver = Chrome(options=chrome_options)
            self.driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
            print("✅ Selenium 브라우저 설정 완료")
        except Exception as e:
            print(f"❌ Selenium 설정 실패: {e}")
            raise

    # ============ 1단계: 전체 기본 목록 크롤링 (추가) ============
    def crawl_basic_list_only(self, max_clicks=150, save_filename='basic_list_full.csv'):
        """1단계: 전체 기본 목록만 크롤링하여 저장 - 안전성 강화 (수정)"""
        print("=== 1단계: 전체 기본 목록 크롤링 시작 ===")

        try:
            self.setup_selenium(headless=True)

            # 접속 및 초기 로딩
            url = f"{self.base_url}/subvention/search"
            print(f"🌐 페이지 접속 중: {url}")
            self.driver.get(url)

            # 로딩 대기
            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            time.sleep(3)

            all_data = []
            current_item_count = 0

            # 첫 번째 페이지 크롤링
            print("📋 첫 페이지 크롤링 중...")
            page_data = self.extract_page_data(start_index=0)
            all_data.extend(page_data)
            current_item_count = len(all_data)
            print(f"📊 현재까지 수집된 아이템: {current_item_count}개")

            # 더보기 버튼 끝까지 클릭
            for click_count in range(max_clicks):
                print(f"\n🔄 더보기 버튼 클릭 시도 {click_count + 1}/{max_clicks}")

                if not self.go_to_next_page():
                    print("✅ 더 이상 더보기 버튼을 찾을 수 없습니다. 전체 목록 크롤링 완료!")
                    break

                # 새로운 아이템들만 크롤링
                new_data = self.extract_page_data(start_index=current_item_count)

                if not new_data:
                    print("ℹ️ 새로운 아이템이 없습니다. 크롤링 종료.")
                    break

                all_data.extend(new_data)
                current_item_count = len(all_data)
                print(f"📊 현재까지 수집된 아이템: {current_item_count}개 (+{len(new_data)}개 추가)")

                # 중간 저장 (500개마다)
                if current_item_count % 500 == 0:
                    temp_filename = f"/content/drive/MyDrive/KB AI CHALLENGE/temp_{save_filename}"
                    temp_df = pd.DataFrame(all_data)
                    temp_df.to_csv(temp_filename, index=False, encoding='utf-8-sig')
                    print(f"💾 중간 저장 완료: {temp_filename} ({current_item_count}개)")

                time.sleep(2)

            # 기본 목록 저장
            if all_data:
                save_path = f"/content/drive/MyDrive/KB AI CHALLENGE/{save_filename}"
                df = pd.DataFrame(all_data)
                df.to_csv(save_path, index=False, encoding='utf-8-sig')
                print(f"\n🎉 1단계 완료! 기본 목록 {len(all_data)}개가 {save_filename}에 저장되었습니다.")

                # 체크포인트 정보 저장
                checkpoint_info = {
                    "total_basic_items": len(all_data),
                    "basic_list_file": save_filename,
                    "created_at": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    "status": "basic_list_completed"
                }
                with open('/content/drive/MyDrive/KB AI CHALLENGE/checkpoint_basic.json', 'w', encoding='utf-8') as f:
                    json.dump(checkpoint_info, f, ensure_ascii=False, indent=2)

                return all_data
            else:
                print("❌ 수집된 데이터가 없습니다.")
                return []

        except Exception as e:
            print(f"❌ 1단계 크롤링 중 오류 발생: {e}")
            print("🔧 해결 방법:")
            print("1. 코랩을 재시작하고 다시 시도")
            print("2. 런타임 유형을 확인 (GPU 사용 시 CPU로 변경)")
            print("3. 크롬 드라이버 버전 확인")
            return []

        finally:
            try:
                if hasattr(self, 'driver'):
                    self.driver.quit()
                    print("🔚 브라우저 종료 완료")
            except:
                pass

    # ============ 2단계: 체크포인트 기반 상세 크롤링 (추가) ============
    def crawl_details_with_checkpoint(self, start_index=0, batch_size=100,
                                    basic_list_file='basic_list_full.csv',
                                    save_prefix='detailed_batch'):
        """2단계: 저장된 기본 목록에서 특정 구간의 상세 정보 크롤링"""

        print(f"=== 2단계: 상세 크롤링 시작 (인덱스 {start_index}~{start_index+batch_size-1}) ===")

        # 기본 목록 파일 읽기
        basic_list_file = f"/content/drive/MyDrive/KB AI CHALLENGE/{basic_list_file.split('/')[-1]}"
        if not os.path.exists(basic_list_file):
            print(f"기본 목록 파일({basic_list_file})이 존재하지 않습니다. 1단계를 먼저 실행하세요.")
            return []

        try:
            basic_df = pd.read_csv(basic_list_file, encoding='utf-8-sig')
            print(f"기본 목록 로드 완료: 총 {len(basic_df)}개 항목")

            # 크롤링할 구간 확인
            end_index = min(start_index + batch_size, len(basic_df))
            if start_index >= len(basic_df):
                print(f"시작 인덱스({start_index})가 전체 데이터 수({len(basic_df)})보다 큽니다.")
                return []

            target_data = basic_df.iloc[start_index:end_index].to_dict('records')
            print(f"크롤링 대상: {len(target_data)}개 항목 (인덱스 {start_index}~{end_index-1})")

        except Exception as e:
            print(f"기본 목록 파일 읽기 오류: {e}")
            return []

        # Selenium 설정
        self.setup_selenium(headless=False)

        detailed_data = []

        try:
            for i, item in enumerate(target_data):
                current_index = start_index + i
                print(f"\n[{current_index+1}/{len(basic_df)}] 상세 정보 크롤링 중: {item['title'][:30]}...")

                # 링크로 직접 접속하여 상세 정보 크롤링
                detailed_item = self.extract_detail_data_from_direct_link(item)
                detailed_data.append(detailed_item)

                # 진행률 표시
                if (i + 1) % 10 == 0:
                    print(f"진행률: {i+1}/{len(target_data)} ({((i+1)/len(target_data)*100):.1f}%)")

                time.sleep(1)  # 서버 부하 방지

            # 배치 결과 저장
            if detailed_data:
                batch_filename = f"/content/drive/MyDrive/KB AI CHALLENGE/{save_prefix}_{start_index}_{end_index-1}.csv"
                df = pd.DataFrame(detailed_data)
                df.to_csv(batch_filename, index=False, encoding='utf-8-sig')
                print(f"\n배치 완료! {len(detailed_data)}개 항목이 {batch_filename}에 저장되었습니다.")


                # 체크포인트 정보 업데이트
                self.update_checkpoint(start_index, end_index-1, batch_filename)

                return detailed_data
            else:
                print("상세 크롤링된 데이터가 없습니다.")
                return []

        except Exception as e:
            print(f"2단계 크롤링 중 오류 발생: {e}")
            return detailed_data  # 부분적으로라도 크롤링된 데이터 반환

        finally:
            self.driver.quit()

    def extract_detail_data_from_direct_link(self, base_data):
        """기본 데이터의 링크로 직접 접속하여 상세 정보 추출 (수정)"""
        detail_data = base_data.copy()

        try:
            # 직접 링크로 접속
            self.driver.get(base_data['link'])

            # 페이지 로딩 대기
            WebDriverWait(self.driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".detail_view_box"))
            )

            # 콘텐츠가 있는 inner 컨테이너가 로드될 때까지 대기
            WebDriverWait(self.driver, 10).until(
                lambda driver: any(
                    inner.find_elements(By.CSS_SELECTOR, "h3")
                    for inner in driver.find_elements(By.CSS_SELECTOR, ".inner")
                )
            )

            time.sleep(2)

            # detail_view_box에서 기본 정보 추출
            detail_data.update(self.extract_basic_info())

            # inner에서 상세 섹션 정보 추출
            detail_data.update(self.extract_detail_sections())

        except Exception as e:
            print(f"상세 페이지 크롤링 중 오류: {e}")
            detail_data.update({
                'category': '',
                'agency': '',
                'detailed_description': '',
                'business_overview': '',
                'eligibility_content': '',
                'application_method': '',
                'attachments': '',
                'contact_info': '',
                'source': ''
            })

        return detail_data

    def update_checkpoint(self, start_index, end_index, batch_filename):
        """체크포인트 정보 업데이트 (추가)"""
        checkpoint_file = '/content/drive/MyDrive/KB AI CHALLENGE/checkpoint_detailed.json'

        # 기존 체크포인트 읽기
        if os.path.exists(checkpoint_file):
            with open(checkpoint_file, 'r', encoding='utf-8') as f:
                checkpoint_data = json.load(f)
        else:
            checkpoint_data = {"completed_batches": []}

        # 새 배치 정보 추가 (batch_filename은 이미 전체 경로)
        batch_info = {
            "start_index": start_index,
            "end_index": end_index,
            "batch_file": batch_filename,
            "completed_at": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

        checkpoint_data["completed_batches"].append(batch_info)
        checkpoint_data["last_completed_index"] = end_index
        checkpoint_data["total_completed"] = len(checkpoint_data["completed_batches"])

        # 체크포인트 저장
        with open(checkpoint_file, 'w', encoding='utf-8') as f:
            json.dump(checkpoint_data, f, ensure_ascii=False, indent=2)

        print(f"체크포인트 업데이트 완료: {start_index}~{end_index}")

    def merge_all_batches(self, save_prefix='detailed_batch', final_filename='final_detailed_data.csv'):
        """모든 배치 파일을 하나로 합치기 (추가)"""
        print("=== 모든 배치 파일 병합 시작 ===")
        base_path = '/content/drive/MyDrive/KB AI CHALLENGE'

        # 배치 파일들 찾기
        batch_files = [f for f in os.listdir(base_path) if f.startswith(save_prefix) and f.endswith('.csv')]

        if not batch_files:
            print("병합할 배치 파일이 없습니다.")
            return None

        batch_files.sort()  # 파일명 순서로 정렬
        print(f"찾은 배치 파일: {len(batch_files)}개")

        all_data = []
        for batch_file in batch_files:
            try:
                full_path = f"{base_path}/{batch_file}"
                df = pd.read_csv(full_path, encoding='utf-8-sig')
                all_data.append(df)
                print(f"{batch_file}: {len(df)}개 항목")
            except Exception as e:
                print(f"{batch_file} 읽기 오류: {e}")

        if all_data:
            final_df = pd.concat(all_data, ignore_index=True)
            final_path = f"{base_path}/{final_filename}"
            final_df.to_csv(final_path, index=False, encoding='utf-8-sig')
            print(f"\n병합 완료! 총 {len(final_df)}개 항목이 {final_path}에 저장되었습니다.")
            return final_df

    def get_checkpoint_status(self):
        """체크포인트 상태 확인 (추가)"""
        print("=== 체크포인트 상태 확인 ===")
        base_path = '/content/drive/MyDrive/KB AI CHALLENGE'

        # 1단계 상태
        basic_checkpoint = f'{base_path}/checkpoint_basic.json'
        if os.path.exists(basic_checkpoint):
            with open(basic_checkpoint, 'r', encoding='utf-8') as f:
                basic_info = json.load(f)
            print(f"1단계 (기본 목록): 완료 - {basic_info['total_basic_items']}개 항목")
        else:
            print("1단계 (기본 목록): 미완료")

        # 2단계 상태
        detailed_checkpoint = f'{base_path}/checkpoint_detailed.json'
        if os.path.exists(detailed_checkpoint):
            with open(detailed_checkpoint, 'r', encoding='utf-8') as f:
                detailed_info = json.load(f)
            print(f"2단계 (상세 크롤링): {detailed_info['total_completed']}개 배치 완료")
            print(f"마지막 완료 인덱스: {detailed_info['last_completed_index']}")

            # 완료된 배치들 표시
            for batch in detailed_info['completed_batches']:
                print(f"  - {batch['start_index']}~{batch['end_index']}: {batch['batch_file']}")
        else:
            print("2단계 (상세 크롤링): 미시작")

    # ============ 기존 메서드들 (수정 없음) ============
    # get_subvention_list_with_selenium, extract_page_data, debug_page_structure 등은 그대로 유지

    def get_subvention_list_with_selenium(self, max_clicks=5):
        """더보기 버튼을 클릭하며 증분 크롤링"""
        self.setup_selenium(headless=False)  # 브라우저 켜기

        try:
            # 접속 및 초기 로딩
            url = f"{self.base_url}/subvention/search"
            self.driver.get(url)

            # 로딩 대기
            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            time.sleep(3)

            all_data = []
            current_item_count = 0

            # 첫 번째 페이지 크롤링 (0번째부터)
            print("첫 페이지 크롤링 중...")
            page_data = self.extract_page_data(start_index=0)
            all_data.extend(page_data)
            current_item_count = len(all_data)
            print(f"현재까지 수집된 아이템: {current_item_count}개")

            # 더보기 버튼 클릭하며 추가 크롤링
            for click_count in range(max_clicks):
                print(f"\n더보기 버튼 클릭 시도 {click_count + 1}/{max_clicks}")

                if not self.go_to_next_page():
                    print("더 이상 더보기 버튼을 찾을 수 없습니다.")
                    break

                # 새로운 아이템들만 크롤링 (이전 개수부터 시작)
                print(f"새로운 아이템 크롤링 중... (시작 인덱스: {current_item_count})")
                new_data = self.extract_page_data(start_index=current_item_count)

                if not new_data:
                    print("새로운 아이템이 없습니다.")
                    break

                all_data.extend(new_data)
                current_item_count = len(all_data)
                print(f"현재까지 수집된 아이템: {current_item_count}개 (+{len(new_data)}개 추가)")

                time.sleep(2)  # 안정화

            print(f"\n크롤링 완료! 총 {len(all_data)}개 아이템 수집")
            return all_data

        except Exception as e:
            print(f"오류 발생: {e}")
            return []

        finally:
            pass

    def extract_page_data(self, start_index=0):
        """지정된 시작 인덱스부터 데이터 추출"""
        data = []

        try:
            print(f"\n=== 데이터 추출 시작 (인덱스 {start_index}부터) ===")
            print(f"페이지 제목: {self.driver.title}")

            # 전체 아이템 가져오기
            items = self.driver.find_elements(By.CSS_SELECTOR, "li.guide_list_item")
            total_items = len(items)
            print(f"페이지 내 전체 항목 수: {total_items}")

            # 시작 인덱스가 전체 아이템 수보다 크면 빈 리스트 반환
            if start_index >= total_items:
                print(f"시작 인덱스({start_index})가 전체 아이템 수({total_items})보다 큽니다.")
                return data

            # 시작 인덱스부터 끝까지만 처리
            items_to_process = items[start_index:]
            print(f"처리할 항목 수: {len(items_to_process)} (인덱스 {start_index} ~ {total_items-1})")

            for i, item in enumerate(items_to_process):
                actual_index = start_index + i  # 실제 인덱스 계산
                try:
                    title = item.find_element(By.CSS_SELECTOR, "h3.guide_list_title").text.strip()
                    # org = item.find_element(By.CSS_SELECTOR, "p.guide_list_desc").text.strip()

                    region = item.find_element(By.XPATH, './/dt[text()="지역"]/following-sibling::dd[1]').text.strip()
                    amount = item.find_element(By.XPATH, './/dt[text()="금액"]/following-sibling::dd[1]').text.strip()
                    deadline = item.find_element(By.XPATH, './/dt[text()="접수 마감일"]/following-sibling::dd[1]').text.strip()

                    link_elem = item.find_element(By.CSS_SELECTOR, "a.guide_list_link")
                    link = link_elem.get_attribute("href")
                    full_text = item.text.strip()

                    data.append({
                        'index': actual_index,  # 디버깅용 인덱스 추가
                        'title': title,
                        # 'organization': org,
                        'region': region,
                        'amount': amount,
                        'deadline': deadline,
                        'link': f"https://mybiz.pay.naver.com{link}" if link and not link.startswith("http") else link,
                        'full_text': full_text[:500],
                        'scraped_at': time.strftime('%Y-%m-%d %H:%M:%S')
                    })

                    print(f"[{actual_index}] 추출 완료: {title[:30]}...")

                except Exception as e:
                    print(f"[{actual_index}] 항목 추출 중 오류: {e}")
                    continue

            print(f"추출 완료: {len(data)}개 항목")

        except Exception as e:
            print(f"데이터 추출 실패: {e}")

        return data

    # 기존의 모든 extract_ 메서드들은 그대로 유지
    # (extract_basic_info, extract_detail_sections, normalize_section_name 등)

    def extract_basic_info(self):
        """detail_view_box에서 기본 정보 추출"""
        basic_info = {}

        try:
            detail_box = self.driver.find_element(By.CSS_SELECTOR, ".detail_view_box")

            # 카테고리 추출
            try:
                category = detail_box.find_element(By.CSS_SELECTOR, ".theme_navy").text.strip()
                basic_info['category'] = category
            except:
                basic_info['category'] = ''

            # 기관명 추출
            try:
                agency = detail_box.find_element(By.CSS_SELECTOR, ".mss_txt").text.strip()
                basic_info['agency'] = agency
            except:
                basic_info['agency'] = ''

            # 상세 설명 추출
            try:
                desc = detail_box.find_element(By.CSS_SELECTOR, ".detail_desc").text.strip()
                basic_info['detailed_description'] = desc
            except:
                basic_info['detailed_description'] = ''

            print(f"기본 정보 추출 완료: 카테고리={basic_info.get('category', 'N/A')}, 기관={basic_info.get('agency', 'N/A')}")

        except Exception as e:
            print(f"기본 정보 추출 중 오류: {e}")
            basic_info = {
                'category': '',
                'agency': '',
                'detailed_description': ''
            }

        return basic_info

    def extract_detail_sections(self):
        """inner 컨테이너에서 각 섹션별 상세 정보 추출 - 올바른 inner 선택"""
        sections = {}

        try:
            # 모든 inner 컨테이너 찾기
            inner_containers = self.driver.find_elements(By.CSS_SELECTOR, ".inner")

            # 실제 콘텐츠가 있는 inner 찾기 (h3 태그가 있는 것)
            content_inner = None
            for inner in inner_containers:
                h3_tags = inner.find_elements(By.CSS_SELECTOR, "h3")
                if h3_tags and any("사업개요" in h3.text for h3 in h3_tags):
                    content_inner = inner
                    break

            if not content_inner:
                print("콘텐츠가 있는 inner 컨테이너를 찾을 수 없습니다.")
                return {}

            print("올바른 inner 컨테이너 찾기 성공")

            # 모든 guide_view_content 요소들 가져오기
            content_elements = content_inner.find_elements(By.CSS_SELECTOR, ".guide_view_content_v2, .guide_view_content")
            print(f"총 content 요소 수: {len(content_elements)}")

            if len(content_elements) == 0:
                print("ERROR: guide_view_content 요소를 찾을 수 없습니다!")
                return {}

            current_section = None
            section_content = []

            for element in content_elements:
                # blank_space 요소는 건너뛰기
                if self.is_blank_space(element):
                    continue

                # h3, h4 태그로 섹션 구분
                headers = element.find_elements(By.CSS_SELECTOR, "h3, h4")

                if headers:
                    # 이전 섹션 저장
                    if current_section and section_content:
                        sections[current_section] = self.clean_section_content(section_content)

                    # 새 섹션 시작
                    header_text = headers[0].text.strip()
                    current_section = self.normalize_section_name(header_text)
                    section_content = []

                    # 헤더와 같은 div에 다른 내용이 있는지 확인
                    element_text = element.text.strip()
                    if element_text != header_text:  # 헤더 외에 다른 내용이 있으면
                        remaining_text = element_text.replace(header_text, '').strip()
                        if remaining_text:
                            section_content.append(remaining_text)
                else:
                    # 섹션 내용 수집
                    text_content = element.text.strip()
                    if text_content and current_section:
                        # 테이블 처리
                        tables = element.find_elements(By.CSS_SELECTOR, "table")
                        if tables:
                            table_text = self.extract_table_content(tables[0])
                            if table_text:
                                section_content.append(table_text)
                        else:
                            section_content.append(text_content)

                        # 첨부파일 링크 처리
                        if current_section == 'attachments':
                            links = element.find_elements(By.CSS_SELECTOR, "a.link.mark")
                            if links:
                                attachment_names = [link.text.strip() for link in links if link.text.strip()]
                                if attachment_names:
                                    section_content.extend(attachment_names)

            # 마지막 섹션 저장
            if current_section and section_content:
                sections[current_section] = self.clean_section_content(section_content)

            print(f"추출된 섹션: {list(sections.keys())}")

        except Exception as e:
            print(f"섹션 추출 중 오류: {e}")

        # 표준화된 컬럼명으로 반환
        return {
            'business_overview': sections.get('business_overview', ''),
            'eligibility_content': sections.get('eligibility_content', ''),
            'application_method': sections.get('application_method', ''),
            'attachments': sections.get('attachments', ''),
            'contact_info': sections.get('contact_info', ''),
            'source': sections.get('source', '')
        }

    def normalize_section_name(self, header_text):
        """섹션 헤더를 표준화된 이름으로 변환 - 수정된 버전"""
        mapping = {
            '사업개요': 'business_overview',
            '지원자격 및 내용': 'eligibility_content',
            '지원 내용': 'eligibility_content',  # h4도 같은 섹션으로 처리
            '신청방법 및 서류': 'application_method',
            '첨부파일': 'attachments',
            '문의처(소관부처)': 'contact_info',
            '문의처': 'contact_info',
            '출처': 'source'
        }
        return mapping.get(header_text, header_text.lower().replace(' ', '_'))

    def clean_section_content(self, content_list):
        """섹션 내용을 정리하여 문자열로 반환 - 수정된 버전"""
        if not content_list:
            return ''

        # 빈 문자열 및 중복 제거, 공백 정리
        cleaned = []
        for item in content_list:
            if item:
                # 다중 공백을 단일 공백으로 변환하고 앞뒤 공백 제거
                cleaned_item = ' '.join(item.split())
                if cleaned_item and cleaned_item not in cleaned:
                    cleaned.append(cleaned_item)

        return '\n\n'.join(cleaned)  # 섹션 내용 간에는 더블 라인브레이크

    def is_blank_space(self, element):
        """blank_space 클래스인지 확인 - 수정된 버전"""
        classes = element.get_attribute('class') or ''

        # blank_space 클래스가 있거나 텍스트가 비어있는 경우
        if 'blank_space' in classes:
            return True

        # 텍스트 내용이 비어있거나 공백만 있는 경우
        text_content = element.text.strip()
        if not text_content or text_content == '\u00a0':  # non-breaking space
            return True

        return False

    def extract_table_content(self, table):
        """테이블 내용을 텍스트로 변환"""
        try:
            rows = table.find_elements(By.CSS_SELECTOR, "tr")
            table_text = []

            for row in rows:
                cells = row.find_elements(By.CSS_SELECTOR, "td, th")
                row_text = " | ".join([cell.text.strip() for cell in cells if cell.text.strip()])
                if row_text:
                    table_text.append(row_text)

            return '\n'.join(table_text)
        except Exception as e:
            print(f"테이블 추출 중 오류: {e}")
            return ""

    def go_to_next_page(self):
        """더보기 버튼 눌러서 다음 콘텐츠 로딩"""
        try:
            # 더보기 버튼 찾기
            more_button = self.driver.find_element(
                By.CSS_SELECTOR,
                "button.btn_guide_more[data-au='search.more_btn']"
            )

            # 버튼이 보이고 클릭 가능한 상태인지 확인
            if more_button.is_displayed() and more_button.is_enabled():
                more_button.click()
                time.sleep(2)  # 콘텐츠 로딩 기다리기
                return True
            else:
                return False

        except Exception as e:
            print(f"더보기 클릭 중 오류: {e}")
            return False

    def save_to_csv(self, data, filename='naver_mybiz_subvention.csv'):
        """데이터를 CSV 파일로 저장"""
        if data:
            df = pd.DataFrame(data)
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            print(f"데이터가 {filename} 파일로 저장되었습니다. (총 {len(data)}건)")
            return df
        else:
            print("저장할 데이터가 없습니다.")
            return None




### 메인 실행 -> 이거 말고

In [ ]:
"""
# ============ 메인 실행 부분 (추가) ============
if __name__ == "__main__":
    crawler = NaverMybizCrawler()

    print("=== 네이버 마이비즈 정책지원금 크롤러 ===")
    print("1. 전체 기본 목록 크롤링 (1단계)")
    print("2. 상세 정보 크롤링 (2단계)")
    print("3. 체크포인트 상태 확인")
    print("4. 모든 배치 파일 병합")
    print("0. 종료")

    while True:
        choice = input("\n실행할 작업을 선택하세요 (0-4): ").strip()

        if choice == "1":
            print("\n=== 1단계: 전체 기본 목록 크롤링 시작 ===")
            max_clicks = input("최대 더보기 클릭 수 (기본값: 150): ").strip()
            max_clicks = int(max_clicks) if max_clicks.isdigit() else 150

            print(f"크롤링 시작... (최대 {max_clicks}번 클릭)")
            crawler.crawl_basic_list_only(max_clicks=max_clicks)

        elif choice == "2":
            print("\n=== 2단계: 상세 정보 크롤링 시작 ===")

            # 기본 목록 파일 확인
            if not os.path.exists('basic_list_full.csv'):
                print("❌ 기본 목록 파일이 없습니다. 먼저 1단계를 실행하세요.")
                continue

            # 기본 목록 총 개수 확인
            basic_df = pd.read_csv('basic_list_full.csv', encoding='utf-8-sig')
            total_items = len(basic_df)
            print(f"📊 총 기본 목록: {total_items}개")

            # 체크포인트 확인
            last_completed = -1
            if os.path.exists('checkpoint_detailed.json'):
                with open('checkpoint_detailed.json', 'r', encoding='utf-8') as f:
                    checkpoint_data = json.load(f)
                last_completed = checkpoint_data.get('last_completed_index', -1)
                print(f"📍 마지막 완료 인덱스: {last_completed}")

            # 시작 인덱스와 배치 크기 입력
            start_index = input(f"시작 인덱스 (기본값: {last_completed + 1}): ").strip()
            start_index = int(start_index) if start_index.isdigit() else (last_completed + 1)

            batch_size = input("배치 크기 (기본값: 100): ").strip()
            batch_size = int(batch_size) if batch_size.isdigit() else 100

            if start_index >= total_items:
                print(f"❌ 시작 인덱스({start_index})가 총 항목 수({total_items})보다 큽니다.")
                continue

            print(f"🚀 상세 크롤링 시작: {start_index}~{min(start_index + batch_size - 1, total_items - 1)}")
            crawler.crawl_details_with_checkpoint(start_index=start_index, batch_size=batch_size)

        elif choice == "3":
            print("\n=== 체크포인트 상태 확인 ===")
            crawler.get_checkpoint_status()

        elif choice == "4":
            print("\n=== 모든 배치 파일 병합 ===")
            final_filename = input("최종 파일명 (기본값: final_detailed_data.csv): ").strip()
            final_filename = final_filename if final_filename else 'final_detailed_data.csv'

            result_df = crawler.merge_all_batches(final_filename=final_filename)
            if result_df is not None:
                print(f"✅ 병합 완료: {final_filename}")

        elif choice == "0":
            print("프로그램을 종료합니다.")
            break

        else:
            print("❌ 잘못된 선택입니다. 0-4 중에서 선택하세요.")
"""


### 직접 시작

    # 파라미터: max_clicks, for문
    # max_clicks = 더보기 버튼 클릭 수 : 즉 30 + (max_clicks*30) 만큼의 글이 로딩됨
    # for문: 글 500개까지 100개씩 배치처리함 (배치마다 저장)
    
    # 1단계로 다 불러오고 2단계에서 start_index 설정해서 체크포인트에서 크롤링 다시시작할수잇음(아마..)
    # 내 구드로 연결해둬서 /content/drive/MyDrive/KB AI CHALLENGE/ -> 이거 ctrl+F 해서 다 바꾸기 + 저장됐는지 꼭확인!!


In [ ]:
if __name__ == "__main__":
    crawler = NaverMybizCrawler()



    # 1단계 : 목록 카드 크롤링
    crawler.crawl_basic_list_only(max_clicks=15)

    # 2단계 : 상세페이지 크롤링
    for i in range(0, 500, 100):  # 0, 100, 200, 300, 400
      crawler.crawl_details_with_checkpoint(start_index=i, batch_size=100)


    # 상태 확인
    crawler.get_checkpoint_status()

    # 최종 병합
    crawler.merge_all_batches()


    # 2단계만 실행 (0-99번째)
    # crawler.crawl_details_with_checkpoint(start_index=0, batch_size=100)
    # crawler.crawl_details_with_checkpoint(start_index=100, batch_size=100)

    # 연속 실행 (여러 배치)
    #for i in range(0, 1000, 100):  # 0, 100, 200, ..., 900
    #  crawler.crawl_details_with_checkpoint(start_index=i, batch_size=100)


=== 1단계: 전체 기본 목록 크롤링 시작 ===
✅ Selenium 브라우저 설정 완료
🌐 페이지 접속 중: https://mybiz.pay.naver.com/subvention/search
📋 첫 페이지 크롤링 중...

=== 데이터 추출 시작 (인덱스 0부터) ===
페이지 제목: 네이버페이 마이비즈
페이지 내 전체 항목 수: 30
처리할 항목 수: 30 (인덱스 0 ~ 29)
[0] 추출 완료: 2025년 중소기업 지식재산 지원 (8월)...
[1] 추출 완료: 2025년 유해물질시험분석 수수료 지원사업 (8월)...
[2] 추출 완료: 경기R&DB센터, 광교비즈니스센터 입주기업 모집...
[3] 추출 완료: 2025년 미래자동차 사업재편 혁신성장 지원사업 지원기...
[4] 추출 완료: [경기] 2025년 팹리스 수요연계 양산지원 사업 양산...
[5] 추출 완료: [경기] 2025년 고용위기 대응 프로젝트 추진계획 수...
[6] 추출 완료: [경남] 김해시 2025년 2차 공공판로 컨설팅 지원사...
[7] 추출 완료: [경북] 2025년 향토뿌리기업 및 산업유산 지정계획 ...
[8] 추출 완료: [광주] 북구 2025년 지식재산 기업 육성을 위한 중...
[9] 추출 완료: [경남] 2025년 베트남 K-Food 팝업스토어 참가...
[10] 추출 완료: (해외전시) 2025년 코스모프로프 미용박람회 참가지원...
[11] 추출 완료: 「'25년 산업특화형 BI 육성 지원 사업」 BI Te...
[12] 추출 완료: 2025년 8월 1차 FTA활용 실무교육 참가기업 모집...
[13] 추출 완료: 2025년 용인 기업 기획전 입점 지원 사업 2차 모집...
[14] 추출 완료: 2025년 반도체기업 HR컨설팅 지원 참여기업 모집 재...
[15] 추출 완료: '고안전 보급형 리튬인산철(LFP) 배터리 상용화 기반...
[16] 추출 완료: 2025년 경기 콘텐츠코리아 랩 콘텐츠 전시 마케팅 지...
[17] 추출 완료: 2025년 하반기 「